In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re

# Aijobsindustry_dataset = pd.read_csv("AIJobsIndustry_Clean.csv")
# Aijobsindustry_dataset

# Dice_us_jobs_dataset = pd.read_csv("Dice_US_jobs_Clean.csv", encoding="cp437")
# Dice_us_jobs_dataset

monster_dataset = pd.read_csv("Monster_Clean.csv")
monster_dataset

# alldata_dataset = pd.read_csv("alldata.csv")
# alldata_dataset


# datasets_to_merge = []


# merged_dataset = pd.concat(Dice_US_jobs_Clean.csv,how="outer")\

csv_in_path1 = os.path.join("alldata.csv")
#csv_in_path2 = os.path.join("AIJobsIndustry_Clean.csv")
#csv_in_path3 = os.path.join("Dice_US_jobs_Clean.csv")
data1 = pd.read_csv(csv_in_path1)
#data2 = pd.read_csv(csv_in_path2)
#data3 = pd.read_csv(csv_in_path3)
data1.head()


In [ ]:
def parser(column):
    word_count = {}
    for entry in column:
        entry = re.sub('[^A-Za-z0-9 ]+', '', str(entry))
#        print(entry)
        try:
            entry = entry.replace(',','')
            word_split = entry.split()
            for word in word_split:
                if word not in word_count:
                    word_count[word] = 1
                else:
                    word_count[word] += 1
        except AttributeError:
            print("Skipping an entry due to bad data")
            continue
    print(word_count)

In [ ]:
desc = data1['description']
parser(desc)

In [ ]:
group_by_state = monster_dataset.groupby("Loc_state")
count_for_state = group_by_state.count()
count_for_state = count_for_state["Position"]
count_for_state